# Проект по SQL 

# Цель
Неоходимо проанализировать базу данных, с информацией о книгах, издательствах, авторах, а также пользовательских обзоров книг, для формулировки ценностного предложение для сервиса чтения книг по подписке, купленного компанией.

# Описание данных
Таблица books
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Подключаемся к базе данных 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
def sql(query):
    return pd.io.sql.read_sql(query, con = engine) 

## Знакомство с данными 

In [4]:
query = '''
select *
from books
limit 5'''
sql(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
select *
from authors
limit 5'''
sql(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
select *
from publishers
limit 5'''
sql(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''
select *
from ratings
limit 5'''
sql(query)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''
select *
from reviews
limit 5'''
sql(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Книги которые вышло после 1 января 2000 года

In [9]:
query = '''
select count(book_id)
from books
where publication_date > '2000-01-01'
'''
sql(query)

,count
0,819


После Января 2000го вышло 819 книг

## Количество обзоров и средняя оценка на каждую книгу 


In [10]:
query = '''
SELECT b.title ,
         count(username) AS number_of_reviews,
         avg(rating) AS avg_rating
FROM books AS b
LEFT JOIN ratings AS r
    ON b.book_id=r.book_id
GROUP BY  b.title
'''
sql(query)




,title,number_of_reviews,avg_rating
0,The Count of Monte Cristo,23,4.217391
1,Count Zero (Sprawl #2),2,2.500000
2,The Botany of Desire: A Plant's-Eye View of th...,2,3.500000
3,The Poisonwood Bible,22,4.363636
4,The Canterbury Tales,6,3.333333
...,...,...,...
994,Of Love and Other Demons,2,4.500000
995,In the Heart of the Sea: The Tragedy of the Wh...,3,3.333333
996,Welcome to Temptation (Dempseys #1),2,5.000000
997,World's End (The Sandman #8),2,4.500000


Самое большое количество обзоров на книгу 'Мемуары гейши', со средней оценкой 4.1

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [11]:
query = '''
select publisher, count (b.book_id)
from publishers as p
left outer join books AS b ON b.publisher_id = p.publisher_id
where b.num_pages > 50
group by publisher
order by count desc
limit 1'''
sql(query)

,publisher,count
0,Penguin Books,42


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [12]:
query = '''
select a.author , avg(r2.rating) as avg_rating, count(r2.rating)
from books b
join ratings r2 
on b.book_id = r2.book_id
join authors a 
on b.author_id = a.author_id 
where b.book_id  in (
                        select bo.book_id
                        from books bo, ratings r 
                        where bo.book_id = r.book_id 
                        group by bo.book_id
                        having count(*) >= 50)
group by a.author 
order by avg_rating desc
limit 10'''
sql(query)


,author,avg_rating,count
0,J.K. Rowling/Mary GrandPré,4.287097,310
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
2,J.R.R. Tolkien,4.246914,162
3,Louisa May Alcott,4.192308,52
4,Rick Riordan,4.080645,62
5,William Golding,3.901408,71
6,J.D. Salinger,3.825581,86
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474,57
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879,66
9,Lois Lowry,3.750000,56


Роулинг поличила наиболее высокую среднюю оценку среди книг с 50 и более оценками.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query = '''
select avg(count)
from 
(select count (review_id)
from reviews
where username in
(select username
from ratings
group by username
HAVING count(rating) > 50)
group by username) as count'''
sql(query)


,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

# Вывод
В ходе анализа базы данных сервиса для чтения книг по подписке, мы получили следующие результаты:

- После января 2000 года вышло 819 книг
- Самое большое количество обзоров на книгу 'Мемуары гейши', со средней оценкой 4.1
- Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц
- Роулинг поличила наиболее высокую среднюю оценку среди книг с 50 и более оценками.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24